In [ ]:
import scipy.sparse as sr
import scipy.sparse.linalg as la
import matplotlib.pyplot as plt


from scipy import io

from opt_operators import *
from algorithms import *
from pd_algorithms import *
from new_pd_algorithms import *
from misc_for_output import *
from gr_pd_algorithms import *

gen = 100
#gen  = np.random.choice(100),对所有的数据使用固定的随机生成器，使得实验具有可重复性

n = 1000
m = 200
s = 10
la = 0.1

np.random.seed(gen)
A = np.random.normal(0,1, (m,n))
np.random.seed(gen)
w = np.random.uniform(-10,10, n)
w[s:] = 0
np.random.seed(gen)
w = np.random.permutation(w)

n = 2000
m = 1000
s = 100
la = 0.1

np.random.seed(gen)
A = np.random.normal(0,1, (m,n))
np.random.seed(gen)
w = np.random.uniform(-10,10, n)

#w = np.random.normal(0,1, n)
w[s:] = 0
np.random.seed(gen)
w = np.random.permutation(w)



# In the paper p = 0.5  or p = 0.9
n = 5000
m = 1000
s = 50
la = 0.1

np.random.seed(gen)
B = np.random.normal(0,1, (m,n))
p = 0.9
A = np.zeros((m,n))
A[:,0] = B[:,0]/np.sqrt(1-p**2)
for j in np.arange(1,n):
    A[:,j] = p*A[:,j-1] + B[:,j]

np.random.seed(gen)
w = np.random.uniform(-10,10, n)
w[s:] = 0
np.random.seed(gen)
w = np.random.permutation(w)

np.random.seed(gen)
nu = np.random.normal(0,0.1, m)
b = A.dot(w) + nu


def f_conj(y):
    return 0.5*(y+b).dot(y+b)

def g(x):
    return la*LA.norm(x,1)

def prox_g(x, rho):
    return prox_norm_1(x,la*rho)

def prox_f_conj(y, rho):
    return (y - rho*b)/(1+rho)

# define energy
def J(x,y):
    t = A.dot(x)-b
    return 0.5* t.dot(t) + la* LA.norm(x,1)

def J1(y,x):
    return J(x,y)


L = np.sqrt(np.max(LA.eigh(A.dot(A.T))[0]))

# starting points
x0 = np.zeros(n)
y0 = -b

# step size for PDAL and PDAL (doesn't require any expensive computation)
tau0 = np.sqrt(m)/LA.norm(A)  #for pdal and npdal

# step size for PDA and NPDA
tau = 1./L
sigma = 1./L

phi = 1.618
sqrt_phi = np.sqrt(phi)
tau1 = tau*sqrt_phi      # for GRPDA
sigma1 = sigma*sqrt_phi    # for GRPDA

prec=1e-10
beta = 400
y1 = y0 + np.random.random(y0.shape) * prec
# need to fix division in case of zero in the denominator
m = LA.norm(y1 - y0) / LA.norm(A.T.dot(y1 - y0))
tau2 = np.sqrt(1.5/beta) * m     # for  GRPDA-L

T = 1e-15
N = 45000
ans1 = pd(J, prox_g, prox_f_conj, A, x0, y0, 0.05*sigma, 20*tau, numb_iter=N)
ans2 = pd_accelerated_primal(J1, prox_f_conj, prox_g, -A.T, y0, x0, tau, sigma, 0.1,   numb_iter=N)
ans3 = pd_linesearch_dual_is_square_norm(J, prox_g, -b, A, x0, y0 , tau0, 1./400, numb_iter=N)

ans4 = pd_Golden(J, prox_g, prox_f_conj, A, x0, y0, 0.05*sigma1, 20*tau1,phi, numb_iter=N,tol=T)
ans5 = pd_accelerated_Golden(J1, prox_f_conj, prox_g,  -A.T, y0, x0,  L, 1, 0.01, numb_iter=N)
ans6 = pd_Golden_linesearch_dual_is_square_norm(J, prox_g, -b, A, x0, y0, tau2, 1./beta, numb_iter=N, tol=T)

ans7 = pd_new(J, prox_g,prox_f_conj, A,  x0, y0,0.3*sigma, 3*tau, numb_iter=N)
ans8 =pd_new_accelerate_dual(J, prox_g, prox_f_conj, A,  x0, y0, 3.3*tau,0.3*sigma,0.01, numb_iter=N)
ans9= pd_new_linesearch_dual_is_square_norm(J, prox_g, -b, A, x0, y0,tau0, 1/10,numb_iter=N)


import matplotlib as mpl
mpl.rc('lines', linewidth=1.5)
mpl.rcParams.update(
    {'font.size': 13, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
mpl.rcParams['xtick.major.pad'] = 2
mpl.rcParams['ytick.major.pad'] = 2

#残差f(Ax)+g(x) - f(Ax^*)-g(x^*)与CPU时间的关系
answers = [ans1, ans2, ans3, ans4,ans5,ans6,ans7,ans8,ans9]
alg = ["PDA","APDA","PDAL","GRPDA","AGRPDA","GRPDAL","NPDA","ANPDA","NPDAL"]

time_stamp =1700
a1,a2 = cut_time(ans1[0],ans1[1], t=time_stamp)
b1,b2 = cut_time(ans2[0],ans2[1], t=time_stamp)
c1,c2 = cut_time(ans3[0],ans3[1], t=time_stamp)

d1,d2 = cut_time(ans4[-1],ans4[0], t=time_stamp)
e1,e2 = cut_time(ans5[-2],ans5[0], t=time_stamp)
f1,f2 = cut_time(ans6[-2],ans6[0], t=time_stamp)

g1,g2 = cut_time(ans7[0],ans7[1], t=time_stamp)
h1,h2 = cut_time(ans8[0],ans8[1], t=time_stamp)
i1,i2 = cut_time(ans9[0],ans9[1], t=time_stamp)

v_min =  min(ans1[1]+ans2[1]+ans3[1]+ans4[0]+ans5[0]+ans6[0]+ans7[1]+ans8[1]+ans9[1])
#使用了find_smallest_energy(answers)函数来找到所有算法中的最小能量值v_min，用于标准化残差。

plt.plot(a1, a2-v_min, 'b', label=alg[0])
plt.plot(b1, b2-v_min, 'k', label=alg[1])
plt.plot(c1, c2-v_min, 'r', label=alg[2])
plt.plot(d1, d2-v_min, 'g', label=alg[3])
plt.plot(e1, e2-v_min, 'pink', label=alg[4])
plt.plot(f1, f2-v_min, 'c', label=alg[5])
plt.plot(g1, g2-v_min, 'y', label=alg[6])
plt.plot(h1, h2-v_min, 'orange', label=alg[7])
plt.plot(i1, i2-v_min, 'm', label=alg[8])

plt.yscale('log')
#设置了y轴为对数刻度（plt.yscale('log')）
plt.xlabel(u'CPU time, seconds  ')
plt.ylabel(u'$\\phi(x)-\\phi_*$')
#添加了x轴和y轴的标签，以及图例（plt.legend()），以帮助解释图表
plt.legend()
#plt.savefig('figures/lasso-.pdf',bbox_inches='tight')
#plt.title(' l1-regularization')
plt.show()
#plt.savefig将图表保存为PDF文件。bbox_inches='tight'参数用于确保保存的图像不包含多余的空白区域。

#plt.clf()
#清除图像"""
